In [3]:
import pandas as pd
import numpy as np

In [4]:
company_df=pd.read_csv('C:/Users/molot/Downloads/Final_proj/Final_proj/companing.csv')
sendings_df=pd.read_csv('C:/Users/molot/Downloads/Final_proj/Final_proj/sendings.csv')
response_df=pd.read_csv('C:/Users/molot/Downloads/Final_proj/Final_proj/response.csv')
sales_df=pd.read_csv('C:/Users/molot/Downloads/Final_proj/Final_proj/sales.csv')
price_df=pd.read_csv('C:/Users/molot/Downloads/Final_proj/Final_proj/price.csv')

company_df['Finish_date']= pd.to_datetime(company_df['Start_date']) + pd.to_timedelta(np.ceil(company_df['response_period']), unit="D")

df_t1=pd.merge(sendings_df,company_df, on=['campaning_SD'])
df_t2=pd.merge(df_t1,response_df, on=['cell_package_sk'])
df_t3=pd.merge(df_t2,sales_df, on=['Id_customer'])

df=pd.merge(df_t3,price_df,left_on=['id_product'], right_on=['product_id'])
sale_price_df=pd.merge(sales_df,price_df,left_on=['id_product'], right_on=['product_id'])
df['yes_or_no']=((df['Date']>df['Start_date']) & (df['Date']<df['Finish_date'])).astype(int)

display(df['response_code'])


0        1
1        1
2        1
3        1
4        1
        ..
99991    3
99992    3
99993    3
99994    3
99995    3
Name: response_code, Length: 99996, dtype: int64

Определить количество людей в каждой группе

In [5]:
#Количество людей в каждой из выборок в каждой компании
count_people_cel=[]
count_price_cel=[]
count_people_ctr=[]
count_price_ctr=[]
prices_cel=[]
prices_ctr=[]
for g in company_df['campaning_SD']:
    groups=df[(df['campaning_SD']==g )&(df['response_code']!=2)].groupby(['campaning_SD','communication_SK','send_name', 'cell_package_sk']).agg({'Id_customer':'unique','Id_customer': 'count','price':'sum', 'yes_or_no': 'sum'}).reset_index()
    display(groups)
    for n in groups['send_name']:
        group_cel=groups[~groups['send_name'].str.contains('CTR')]
        group_ctr=groups[groups['send_name'].str.contains('CTR')]
    #display(group_cel)
    count_price_cel.append(group_cel['yes_or_no'].tolist())
    count_people_cel.append(group_cel['Id_customer'].tolist())
    count_price_ctr.append(group_ctr['yes_or_no'].tolist())
    count_people_ctr.append(group_ctr['Id_customer'].tolist())
    prices_cel.append(group_cel['price'].tolist())
    prices_ctr.append(group_ctr['price'].tolist())

print("Количество покупок в каждой целевой группе: ", count_price_cel,"\n",f"Количество людей в каждой целевой группе: {count_people_cel} \n")
print("Количество покупок в каждой контрольной группе: ", count_price_ctr,"\n",f"Количество людей в каждой контрольной группе: {count_people_ctr}")
#col_B=df[df['response_code']==3]['Id_customer'].unique()
#print("Количество пользователей в целевой группе:",len(col_A),"\n","Количество пользователей в контрольной группе:",len(col_B))


,campaning_SD,communication_SK,send_name,cell_package_sk,Id_customer,price,yes_or_no
0,camp_3581,808600,23_06_2023_win_prize_A,118512,11922,1669965,2496
1,camp_3581,808600,23_06_2023_win_prize_CTR_A,118513,3111,441320,672
2,camp_3581,959200,23_06_2023_win_prize_B,118514,16947,2385970,3576
3,camp_3581,959200,23_06_2023_win_prize_CTR_B,118515,5710,808570,1222


,campaning_SD,communication_SK,send_name,cell_package_sk,Id_customer,price,yes_or_no
0,camp_35367,335700,28_06_2023_sale_5%,118516,37286,5253535,7862
1,camp_35367,335700,28_06_2023_sale_5%_CTR,118517,4470,628745,922


,campaning_SD,communication_SK,send_name,cell_package_sk,Id_customer,price,yes_or_no
0,camp_3512,450725,03_07_2013_special_offer,118518,6144,870415,1354
1,camp_3512,450725,03_07_2013_special_offer_CTR,118519,684,93845,130


Количество покупок в каждой целевой группе:  [[2496, 3576], [7862], [1354]] 
 Количество людей в каждой целевой группе: [[11922, 16947], [37286], [6144]] 

Количество покупок в каждой контрольной группе:  [[672, 1222], [922], [130]] 
 Количество людей в каждой контрольной группе: [[3111, 5710], [4470], [684]]


# Вывод:
## Компания camp_3581. Группа А
В целевой группе, в группе исследования А компании camp_3581 покупку в указанный период совершило 2496 человек из 11922. <br>
В контрольной группе покупку в указанный период совершило 672 человека из 3111. <br>
## Компания camp_3581. Группа B
В целевой группе, в группе исследования В компании camp_3581 покупку в указанный период совершило 3576 человек из 16947. <br>
В контрольной группе покупку в указанный период совершило 1222 человека из 5710. <br>
## Компания camp_35367
В целевой группе компании camp_35367 покупку в указанный период совершило 9510 человек из 44966. <br>
В контрольной группе покупку в указанный период совершило 922 человека из 4470. <br>
## Компания camp_3512
В целевой группе компании camp_3512 покупку в указанный период совершило 1354 человек из 6144. <br>
В контрольной группе покупку в указанный период совершило 130 человека из 684. <br>

Отклик

In [6]:
# Средняя прибыль от одного платящего пользователя ARPPU
ARPPU_1A_cel=round(prices_cel[0][0]/count_price_cel[0][0],3)
ARPPU_1B_cel=round(prices_cel[0][1]/count_price_cel[0][1],3)
ARPPU_1A_ctr=round(prices_ctr[0][0]/count_price_ctr[0][0],3)
ARPPU_1B_ctr=round(prices_ctr[0][1]/count_price_ctr[0][1],3)

otklic_1A_cel=round(count_price_cel[0][0]/count_people_cel[0][0],3)
otklic_1A_ctr=round(count_price_ctr[0][0]/count_people_ctr[0][0],3)
otklic_1B_cel=round(count_price_cel[0][1]/count_people_cel[0][1],3)
otklic_1B_ctr=round(count_price_ctr[0][1]/count_people_ctr[0][1],3)
d_otrklic_1A=round(otklic_1A_cel-otklic_1A_ctr,3)
d_otrklic_1B=round(otklic_1B_cel-otklic_1B_ctr,3)
print(f'''ARPPU компании camp_3581 группы А составляет:\n целевая {round(ARPPU_1A_cel,2)}\n отклик: {round(otklic_1A_cel*100,2)} \n контрольная: {round(ARPPU_1A_ctr,2)}\n отклик: {round(otklic_1A_ctr*100,2)}\n
группы В: \n целевая {round(ARPPU_1B_cel,2)}\n отклик: {round(otklic_1B_cel*100,2)}\n контрольная: {round(ARPPU_1B_ctr,2)}\nотклик:{round(otklic_1B_ctr*100,2)}\n\n''')

ARPPU_2_cel=prices_cel[1][0]/count_price_cel[1][0]
ARPPU_2_ctr=prices_ctr[1][0]/count_price_ctr[1][0]
otklic_2_ctr=count_price_ctr[1][0]/count_people_ctr[1][0]
otklic_2_cel=count_price_cel[1][0]/count_people_cel[1][0]
d_otrklic_2=round(otklic_2_cel-otklic_2_ctr,3)

print(f'''ARPPU компании camp_35367:\n целевая {round(ARPPU_2_cel,2)}\n отклик: {round(otklic_2_cel*100,2)}\n контрольная: {round(ARPPU_2_ctr,2)}\n отклик: {round(otklic_2_ctr*100,2)}\n\n''')

ARPPU_3_cel=prices_cel[2][0]/count_price_cel[2][0]
ARPPU_3_ctr=prices_ctr[2][0]/count_price_ctr[2][0]
otklic_3_ctr=count_price_ctr[2][0]/count_people_ctr[2][0]
otklic_3_cel=count_price_cel[1][0]/count_people_cel[1][0]
d_otrklic_3=round(otklic_3_cel-otklic_3_ctr,3)
print(f'''ARPPU компании camp_3512:\n целевая {round(ARPPU_3_cel,2)}\n отклик: {round(otklic_3_cel*100,2)} \n контрольная: {round(ARPPU_3_ctr,2)}\n отклик: {round(otklic_3_ctr*100,2)}''')
print(f'''Дельта отклика 1A: {d_otrklic_1A}
Дельта отклика 1B: {d_otrklic_1B}
Дельта отклика 2: {d_otrklic_2}
Дельта отклика 3: {d_otrklic_3}''')

ARPPU компании camp_3581 группы А составляет:
 целевая 669.06
 отклик: 20.9 
 контрольная: 656.73
 отклик: 21.6

группы В: 
 целевая 667.22
 отклик: 21.1
 контрольная: 661.68
отклик:21.4


ARPPU компании camp_35367:
 целевая 668.22
 отклик: 21.09
 контрольная: 681.94
 отклик: 20.63


ARPPU компании camp_3512:
 целевая 642.85
 отклик: 21.09 
 контрольная: 721.88
 отклик: 19.01
Дельта отклика 1A: -0.007
Дельта отклика 1B: -0.003
Дельта отклика 2: 0.005
Дельта отклика 3: 0.021


In [22]:
cols=[]
companys_yon=df[(df['response_code']!=2)].groupby(['campaning_SD','communication_SK']).agg({'Id_customer':'unique','Id_customer': 'count','price':'sum', 'yes_or_no':'sum'}).reset_index()
cols=companys_yon['yes_or_no'].tolist()
cols.reverse()
n1=cols[0]*d_otrklic_1A
n2=cols[1]*d_otrklic_1B
n3=cols[2]*d_otrklic_2
n4=cols[3]*d_otrklic_3
print(f"Количество привлеченных пользователей: {n1}, {n2}, {n3}, {n4}")
print(f"Полученная выручка: {n1*ARPPU_1A_cel}, {n2*ARPPU_1B_cel}, {n3*ARPPU_2_cel}, {n4*ARPPU_3_cel}")

Количество привлеченных пользователей: -33.586, -9.504, 43.92, 31.164
Полученная выручка: -22470.914816, -6341.239871999999, 29348.162961078604, 20033.6876366322


# Вывод
* Не стоит добовлять нововведение в компании camp_3581 в обоих исследованиях, так как при проведении анализа выяснили, что компания понесла убытки.
* Компания camp_35367 принесла больше всего дополнительной выручки, а именно 29348 денежных едениц
* Компаниям camp_35367 и camp_3512 стоит ввести нововведение в виде отправки писем, так как это принесло компании дополнительную прибыль